# Create Cluster: RDB
This notebook will create (start) an RDB cluster.

In [ ]:
%%html
<style>
table {float:left}
</style>

## Setup

### Node Types
|Type|Mem (GB)|vCPUs|
|:---|---:|---:|
|kx.s.large|12|2|
|kx.s.xlarge|27|4|
|kx.s.2xlarge|54|8|
|kx.s.4xlarge|108|16|
|kx.s.8xlarge|216|32|
|kx.s.16xlarge|432|64|
|kx.s.32xlarge|864|128|


In [ ]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env import *

# Managed KX Database and Cluster names to create
DB_NAME="welcomedb"

SEC_THREADS='8'
CLUSTER_NAME=f"RDB_{DB_NAME}"

# Cluster Settings
CODEBASE="code"
S3_CODE_PATH="code"

CAPACITY_CONFIG={ 'nodeCount': 1, 'nodeType': 'kx.s.2xlarge'}
SAVEDOWN_CONFIG={ 'type': 'SDS01', 'size':200 }
DATABASE_CONFIG=[{ 'databaseName': DB_NAME }]
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }

INIT_SCRIPT=f'{CODEBASE}/init.q'
CMD_ARGS=[
    { 'key': 's', 'value': SEC_THREADS }, 
    { 'key': 'dbname', 'value': DB_NAME}, 
    { 'key': 'codebase', 'value': CODEBASE} 
]

In [ ]:
# triggers credential get
session=None

try:
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS sessio: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

## Create Cluster

In [ ]:
resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=CLUSTER_NAME,
    clusterType='RDB',
    releaseLabel = '1.0',
    databases=DATABASE_CONFIG,
    capacityConfiguration=CAPACITY_CONFIG,
    savedownStorageConfiguration=SAVEDOWN_CONFIG,
    clusterDescription="Created with create_cluster_RDB notebook",
    code=CODE_CONFIG,
    initializationScript=INIT_SCRIPT,
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    commandLineArguments=CMD_ARGS,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' }
)

In [ ]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, show_wait=True)
print()
print("** DONE **")

In [ ]:
# Give permissions time to propogate after cluster creation....
time.sleep(60)

In [ ]:
conn_str = get_kx_connection_string(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, userName=KDB_USERNAME, boto_session=session)
print(conn_str)

In [ ]:
cdf = get_clusters(client, ENV_ID)

display(cdf)

In [ ]:
print( f"Last Run: {datetime.datetime.now()}" )